In [1]:
# import ipykernel
import pandas as pd

In [ ]:
# Load data
# Clean data
# Scale data

In [ ]:
# Categorical

# Numerical

# Get Data

In [3]:
import os
import zipfile
# from prefect import flow, task
from kaggle.api.kaggle_api_extended import KaggleApi

KAGGLE_USERNAME = os.getenv("KAGGLE_USERNAME")
KAGGLE_KEY = os.getenv("KAGGLE_KEY")

# @task
def get_kaggle_data():
    """
    Download kaggle data
    """
    api = KaggleApi()
    api.authenticate()
    api.competition_download_files(
        competition="playground-series-s4e8/test,csv",
        path='data/data.zip',
    )
    with zipfile.ZipFile('data/data.zip', 'r') as zip_ref:
        zip_ref.extractall('./data/')



# Preprocess Data

In [1]:
!pip install scikit-learn -q

In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler, FunctionTransformer
from sklearn.compose import make_column_selector, make_column_transformer, ColumnTransformer

df = pd.read_csv('data/train.csv')
# og_df['stem-root'].isna().sum()

In [19]:
cat=df.select_dtypes(include='object').columns.to_list()
cat.remove('class')
cat

['cap-shape',
 'cap-surface',
 'cap-color',
 'does-bruise-or-bleed',
 'gill-attachment',
 'gill-spacing',
 'gill-color',
 'stem-root',
 'stem-surface',
 'stem-color',
 'veil-type',
 'veil-color',
 'has-ring',
 'ring-type',
 'spore-print-color',
 'habitat',
 'season']

In [20]:
num=df.select_dtypes(exclude='object').columns.to_list()
num.remove('id')
num

['cap-diameter', 'stem-height', 'stem-width']

In [21]:
dataset = pd.read_csv('data/train.csv')
preprocessor = Preprocessor(dataset)

In [22]:
X, y = preprocessor.clean()

In [27]:
for col in X[cat].columns:
    print(f"'{col}': {X[col].unique().tolist()}")

'cap-shape': ['f', 'x', 'p', 'b', 'o', 'c', 's', 'd', 'e', 'n', 'unknown', 'w', 'k', 'l', 't', 'g', 'z', 'a', 'r', 'u', 'y', 'i', 'm', 'h'],
'cap-surface': ['s', 'h', 'y', 'l', 't', 'e', 'g', 'unknown', 'd', 'i', 'w', 'k', 'f', 'n', 'r', 'o', 'a', 'u', 'z', 'p', 'b', 'm', 'x', 'c'],
'cap-color': ['u', 'o', 'b', 'g', 'w', 'n', 'e', 'y', 'r', 'p', 'k', 'l', 'i', 'h', 'd', 's', 'a', 'f', 'unknown', 'c', 'x', 'm', 'z', 't'],
'does-bruise-or-bleed': ['f', 't', 'd', 'unknown', 'w', 'o', 'b', 'x', 'p', 'g', 'y', 'r', 'a', 'l', 'i', 'c', 'n', 'z', 's', 'k', 'h', 'e', 'u'],
'gill-attachment': ['a', 'x', 's', 'd', 'e', 'unknown', 'f', 'p', 'l', 'm', 'b', 'n', 'g', 'i', 'u', 't', 'o', 'c', 'w', 'k', 'r', 'h', 'z', 'y'],
'gill-spacing': ['c', 'unknown', 'd', 'f', 'x', 'b', 'a', 'k', 'e', 'y', 's', 'p', 't', 'i', 'w', 'h', 'l', 'r', 'n', 'g'],
'gill-color': ['w', 'n', 'g', 'k', 'y', 'f', 'p', 'o', 'b', 'u', 'e', 'r', 'd', 't', 'unknown', 'z', 'h', 'x', 's', 'c', 'm', 'l', 'a', 'i'],
'stem-root': ['

In [6]:
X_preprocess = preprocessor.preprocess_features(X)

In [18]:
def label_encode(X):
    encoder = LabelEncoder()
    return X.apply(encoder.fit_transform)
    
num_pipeline=make_pipeline(
    StandardScaler()
)
cat_pipeline=make_pipeline(
    FunctionTransformer(func=label_encode, validate=False, feature_names_out='one-to-one'),
)
    
preprocessing=ColumnTransformer([
    ("num", num_pipeline, num),
    ("cat", cat_pipeline, cat),
   ])

preprocessing_pipeline=make_pipeline(preprocessing)

X_transformed=preprocessing_pipeline.fit_transform(X)
feature_names=list(preprocessing_pipeline[0].get_feature_names_out())
X=pd.DataFrame(data=X_transformed, columns=feature_names)

In [7]:
X_train, X_test, y_train, y_test = preprocessor.split(X_preprocess, y, .2)

In [17]:
class Preprocessor():

    def __init__(self, dataset):
        self.dataset=dataset
        self.cat=self.dataset.select_dtypes(include='object').columns.to_list()
        self.cat.remove('class')

        self.num=self.dataset.select_dtypes(exclude='object').columns.to_list()
        self.num.remove('id')

    def clean(self):

        self.dataset=(
            self.dataset
            .assign(**{
                cat_col: lambda x, col=cat_col: np.where(
                    ~x[col].isna() & x[col].str.match('^[a-zA-Z]$'),
                    x[col],
                    "unknown") for cat_col in self.cat})
            .assign(**{
                num_col: self.dataset[num_col].fillna(0)
                for num_col in self.num})
            .assign(**{'class': self.dataset['class'].map({'e': 0, 'p': 1})})
           )

        X, y=(self.dataset.drop(columns=['class', 'id']),
              self.dataset['class'])

        return X, y

    def preprocess_features(self, X):
        def label_encode(X):
            encoder = LabelEncoder()
            return X.apply(encoder.fit_transform)
            
        num_pipeline=make_pipeline(
            StandardScaler()
        )
        cat_pipeline=make_pipeline(
            FunctionTransformer(func=label_encode, validate=False, feature_names_out='one-to-one'),
        )
            
        preprocessing=ColumnTransformer([
            ("num", num_pipeline, self.num),
            ("cat", cat_pipeline, self.cat),
           ])

        preprocessing_pipeline=make_pipeline(preprocessing)

        X_transformed=preprocessing_pipeline.fit_transform(X)
        feature_names=list(preprocessing_pipeline[0].get_feature_names_out())
        X=pd.DataFrame(data=X_transformed, columns=feature_names)

        return X
    
    def split(self, X, y, test_size):
        
        X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                            test_size=test_size,
                                                            random_state=42,
                                                            stratify=y)

        return X_train, X_test, y_train, y_test

# Train Model

In [8]:
!pip install lightgbm -q


^C


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\rbbel\\.virtualenvs\\MLOps-6AcC-k4j\\Lib\\site-packages\\catboost\\_catboost.pyd'
Check the permissions.



In [12]:
!pip install catboost

  Using cached catboost-1.2.5-cp39-cp39-win_amd64.whl.metadata (1.2 kB)
Using cached catboost-1.2.5-cp39-cp39-win_amd64.whl (101.2 MB)


In [11]:
!pip install hyperopt -q

In [16]:
# ensemble
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

# hypertuning
import mlflow
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

# eval
from sklearn.metrics import f1_score, log_loss, roc_auc_score, precision_score, recall_score, matthews_corrcoef

In [26]:
class Modeller():
    def __init__(self, X_train, y_train, X_test, y_test, experiment_name, mlflow_tracking_uri="http://localhost:5000"):
        self.X_train = X_train
        self.y_train = y_train
        self.X_test = X_test
        self.y_test = y_test
        self.mlflow_tracking_uri = mlflow_tracking_uri
        self.experiment_name = experiment_name


    def hypertune(self):
        mlflow.set_tracking_uri(self.mlflow_tracking_uri)
        mlflow.set_experiment(self.experiment_name)
        all_scores={}

        def search_fn(params):
            with mlflow.start_run():
                mlflow.set_tag('person','rchll') # Change name
                mlflow.autolog()
                
                classifier_type = params["type"]
                mlflow.set_tag('model', classifier_type)
                del params["type"]
                if classifier_type == "xgb":
                    model = XGBClassifier(**params)
                elif classifier_type == "lgbm":
                    model = LGBMClassifier(**params)
                elif classifier_type == "catboost":
                    model = LGBMClassifier(**params)

                model.fit(self.X_train, self.y_train)
                y_pred=model.predict(self.X_test)
                
                accuracy=model.score(self.X_test,self.y_test)
                precision=precision_score(self.y_test, y_pred)
                recall=recall_score(self.y_test,y_pred)
                f1=f1_score(self.y_test, y_pred)
                roc=roc_auc_score(self.y_test, y_pred)
                logloss=log_loss(self.y_test, y_pred)
                mcc=matthews_corrcoef(self.y_test,y_pred)
                
                metrics={
                    'accuracy':accuracy,
                    'precision':precision,
                    'recall':recall,
                    'f1':f1,
                    'roc_auc_score': roc,
                    'log_loss': logloss,
                    'mcc':mcc
                }
    
                mlflow.log_metrics(metrics)
    
                return {"loss": -f1, "status": STATUS_OK}

        search_space =  hp.choice(
            "classifier_type",
            [
                {
                    'type': 'xgb',
                    'n_estimators': scope.int(hp.uniform('xgb.n_estimators', 100, 1000)),
                    'max_depth': scope.int(hp.quniform('xgb.max_depth', 10, 100, 1)),
                    'learning_rate': hp.loguniform('xgb.learning_rate', -7, -2),
                    'min_child_weight': hp.loguniform('xgb.min_child_weight', -1, 5),
                    'random_state': 42,
                },
                {
                    "type": "lgbm",
                    'n_estimators': scope.int(hp.uniform('lgbm.n_estimators', 100, 1000)),
                    'learning_rate': hp.loguniform('lgbm.learning_rate', -7, -2),
                    'num_leaves': hp.choice('lgbm.num_leaves', [31, 50, 70]),
                    'max_depth': scope.int(hp.quniform('lgbm.max_depth', 10, 100, 1)),
                    'min_child_samples': hp.choice('lgbm.min_child_samples', [20, 30, 40]),
                    'subsample': hp.uniform('lgbm.subsample', 0.7, 1.0),
                    'colsample_bytree': hp.uniform('lgbm.catboost.colsample_bytree', 0.7, 1.0)
                },
                {
                    "type": "catboost",
                    'iterations': hp.choice('catboost.iterations', [100, 200]),
                    'learning_rate': hp.loguniform('catboost.learning_rate', -7, -2),
                    'depth': hp.choice('catboost.depth', [6, 8, 10]),
                    'l2_leaf_reg': hp.choice('catboost.l2_leaf_reg', [1, 3, 5]),
                    'subsample': hp.uniform('catboost.subsample', 0.7, 1.0),
                    'colsample_bylevel': hp.uniform('catboost.colsample_bylevel', 0.7, 1.0)
                },
            ],
        )
        trials=Trials()
        best_result = fmin(
            fn=search_fn,
            space=search_space,
            algo=tpe.suggest,
            max_evals=10,
            trials=trials
        )

    def register_best_model(self):
        client = MlflowClient(tracking_uri=self.mlflow_tracking_uri)
        experiment = client.get_experiment_by_name(self.experiment_name)
        best_run = client.search_runs(
            experiment_ids=experiment.experiment_id,
            run_view_type=ViewType.ACTIVE_ONLY,
            max_results=1,
            order_by=["metrics.f1 DESC"],
        )[0]
        # register the best model
        run_id = best_run.info.run_id
        model_uri = f"runs:/{run_id}/model"
        model_details = mlflow.register_model(model_uri=model_uri, name=self.experiment_name)

In [ ]:
mlflow.lo

In [27]:
experiment_name = "trial"
mlflow_tracking_uri = "http://localhost:5000"
model = Modeller(X_train, y_train, X_test, y_test, experiment_name, mlflow_tracking_uri)

In [29]:
model.hypertune()

[LightGBM] [Info] Number of positive: 1364317, number of negative: 1129239                                                                                                   
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.112591 seconds.                                                                      
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 968                                                                                                                                             
[LightGBM] [Info] Number of data points in the train set: 2493556, number of used features: 20                                                                               
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.547137 -> initscore=0.189110                                                                                               
[LightGB

2024/08/17 17:27:18 INFO mlflow.tracking._tracking_service.client: 🏃 View run gifted-hen-285 at: http://localhost:5000/#/experiments/2/runs/0c4065b30cdc484b80abd81e7a24bdaf.

2024/08/17 17:27:18 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/2.



[LightGBM] [Warning] Unknown parameter: colsample_bylevel                                                                                                                    
[LightGBM] [Warning] Unknown parameter: depth                                                                                                                                
[LightGBM] [Warning] Unknown parameter: iterations                                                                                                                           
[LightGBM] [Warning] Unknown parameter: l2_leaf_reg                                                                                                                          
[LightGBM] [Warning] Unknown parameter: colsample_bylevel                                                                                                                    
[LightGBM] [Warning] Unknown parameter: depth                                                                                     

2024/08/17 17:27:41 INFO mlflow.tracking._tracking_service.client: 🏃 View run aged-fowl-910 at: http://localhost:5000/#/experiments/2/runs/76f2ba676b71431e8e63bf8b58f4e50e.

2024/08/17 17:27:41 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/2.



[LightGBM] [Info] Number of positive: 1364317, number of negative: 1129239                                                                                                   
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.080445 seconds.                                                                      
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 968                                                                                                                                             
[LightGBM] [Info] Number of data points in the train set: 2493556, number of used features: 20                                                                               
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.547137 -> initscore=0.189110                                                                                               
[LightGB

...

 80%|██████████████████████████████████████████████████████████████████████████████████▍                    | 8/10 [13:12<03:11, 95.73s/trial, best loss: -0.992667713577058]

2024/08/17 17:42:45 INFO mlflow.tracking._tracking_service.client: 🏃 View run adaptable-crab-342 at: http://localhost:5000/#/experiments/2/runs/4631fa1fafe3410889f649328857d35b.

2024/08/17 17:42:45 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/2.



 90%|███████████████████████████████████████████████████████████████████████████████████████████▊          | 9/10 [16:49<02:13, 133.64s/trial, best loss: -0.992667713577058]

2024/08/17 17:46:13 INFO mlflow.tracking._tracking_service.client: 🏃 View run charming-midge-723 at: http://localhost:5000/#/experiments/2/runs/bd6b0cc199704ab89d439152978fe3fd.

2024/08/17 17:46:13 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/2.



100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [20:17<00:00, 121.74s/trial, best loss: -0.992667713577058]


In [21]:
mlflow.end_run()

2024/08/17 17:24:10 INFO mlflow.tracking._tracking_service.client: 🏃 View run zealous-bird-419 at: http://localhost:5000/#/experiments/2/runs/8a5fa9fd8b4a4fca875b337414d54561.
2024/08/17 17:24:10 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/2.


In [33]:
from mlflow import MlflowClient
from mlflow.entities import ViewType
model.register_best_model()

Successfully registered model 'trial'.
2024/08/17 17:59:03 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: trial, version 1
Created version '1' of model 'trial'.


# Model Registry

In [35]:
import os
import mlflow

In [37]:
MLFLOW_TRACKING_URI = 'http://localhost:5000'
EXPERIMENT_NAME = 'mushroom-exp'
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
model_name = EXPERIMENT_NAME
model_version = 'latest'
os.makedirs('app/model', exist_ok=True)
model = mlflow.pyfunc.load_model(model_uri=f"models:/{model_name}/{model_version}", dst_path = "app/model")

C:\Users\rbbel\.virtualenvs\MLOps-6AcC-k4j\lib\site-packages\mlflow\store\artifact\utils\models.py:32: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest = client.get_latest_versions(name, None if stage is None else [stage])


In [39]:
model.predict()

TypeError: predict() missing 1 required positional argument: 'data'

In [3]:
os.makedirs(b2, exist_ok=True)

NameError: name 'b2' is not defined

In [9]:
MLFLOW_TRACKING_URI = 'http://localhost:5000'
EXPERIMENT_NAME = 'mushroom-exp'
def load_model_artifacts_from_mlflow(MLFLOW_TRACKING_URI, EXPERIMENT_NAME):
    mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)    
    model_name = EXPERIMENT_NAME
    model_version = 'latest'
    os.makedirs('app/model', exist_ok=True)
    model = mlflow.pyfunc.load_model(model_uri=f"models:/{model_name}/{model_version}", dst_path = "./app/model")
    with open('app/model/preprocessing_pipeline/preprocessing_pipeline.pkl', 'rb') as f:
        preprocessing_pipeline=pickle.load(f)
    return model, preprocessing_pipeline

In [10]:
load_model_artifacts_from_mlflow(MLFLOW_TRACKING_URI, EXPERIMENT_NAME)

C:\Users\rbbel\.virtualenvs\MLOps-6AcC-k4j\lib\site-packages\mlflow\store\artifact\utils\models.py:32: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest = client.get_latest_versions(name, None if stage is None else [stage])


mlflow.pyfunc.loaded_model:
  artifact_path: model
  flavor: mlflow.xgboost
  run_id: 8a4d71125c52422590f3e6a6601d218c

In [49]:
mlflow.get_artifact_uri(artifact_path=f"models:/{EXPERIMENT_NAME}/latest", )

's3://mlflow-bucket-54/mlflow/0/4b12dc07279d4127af1dc5e8f31b20fc/artifacts/models:/mushroom-exp/latest'

In [41]:
mlflow.get_artifact_uri(artifact_path="preprocessing_pipeline/preprocessing_pipeline.pkl")

's3://mlflow-bucket-54/mlflow/0/4b12dc07279d4127af1dc5e8f31b20fc/artifacts/preprocessing_pipeline/preprocessing_pipeline.pkl'

In [39]:
mlflow.artifacts.get_artifact_repository(artifact_uri=f"models:/{EXPERIMENT_NAME}/latest").

C:\Users\rbbel\.virtualenvs\MLOps-6AcC-k4j\lib\site-packages\mlflow\store\artifact\utils\models.py:32: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest = client.get_latest_versions(name, None if stage is None else [stage])


<method-wrapper '__str__' of method object at 0x00000225DE19E440>

In [23]:
mlflow.artifacts.download_artifacts(artifact_uri="s3://mlflow-bucket-54/mlflow/1/8a4d71125c52422590f3e6a6601d218c/artifacts/model/")

'C:\\Users\\rbbel\\AppData\\Local\\Temp\\tmpf4ik4sk_\\'

In [47]:
mlflow.artifacts.list_artifacts(artifact_uri="s3://mlflow-bucket-54/mlflow/1/8a4d71125c52422590f3e6a6601d218c/artifacts/model/")

[<FileInfo: file_size=2017, is_dir=False, path='MLmodel'>,
 <FileInfo: file_size=224, is_dir=False, path='conda.yaml'>,
 <FileInfo: file_size=20527757, is_dir=False, path='model.xgb'>,
 <FileInfo: file_size=None, is_dir=True, path='preprocessing_pipeline'>,
 <FileInfo: file_size=120, is_dir=False, path='python_env.yaml'>,
 <FileInfo: file_size=105, is_dir=False, path='requirements.txt'>]

# Dummy data

In [2]:
import pandas as pd
import random

In [3]:
data = {
    'cap-shape': random.choice([
        'f', 'x', 'p', 'b', 'o', 'c', 's', 'd', 'e', 'n', 'unknown', 
        'w', 'k', 'l', 't', 'g', 'z', 'a', 'r', 'u', 'y', 'i', 'm', 'h'
        ]),
    'cap-surface': random.choice([
        's', 'h', 'y', 'l', 't', 'e', 'g', 'unknown', 'd', 'i', 'w', 
        'k', 'f', 'n', 'r', 'o', 'a', 'u', 'z', 'p', 'b', 'm', 'x', 'c'
        ]),
    'cap-color': random.choice([
        'u', 'o', 'b', 'g', 'w', 'n', 'e', 'y', 'r', 'p', 'k', 'l', 
        'i', 'h', 'd', 's', 'a', 'f', 'unknown', 'c', 'x', 'm', 'z', 't'
        ]),
    'does-bruise-or-bleed': random.choice([
        'f', 't', 'd', 'unknown', 'w', 'o', 'b', 'x', 'p', 'g', 'y', 
        'r', 'a', 'l', 'i', 'c', 'n', 'z', 's', 'k', 'h', 'e', 'u'
        ]),
    'gill-attachment': random.choice([
        'a', 'x', 's', 'd', 'e', 'unknown', 'f', 'p', 'l', 'm', 'b', 
        'n', 'g', 'i', 'u', 't', 'o', 'c', 'w', 'k', 'r', 'h', 'z', 'y'
        ]),
    'gill-spacing': random.choice([
        'c', 'unknown', 'd', 'f', 'x', 'b', 'a', 'k', 'e', 'y', 's', 
        'p', 't', 'i', 'w', 'h', 'l', 'r', 'n', 'g'
        ]),
    'gill-color': random.choice([
        'w', 'n', 'g', 'k', 'y', 'f', 'p', 'o', 'b', 'u', 'e', 'r', 
        'd', 't', 'unknown', 'z', 'h', 'x', 's', 'c', 'm', 'l', 'a', 'i'
        ]),
    'stem-root': random.choice([
        'unknown', 'b', 'c', 'r', 's', 'f', 'y', 'o', 'k', 'd', 'n', 
        'w', 'u', 'p', 'x', 'i', 'a', 't', 'm', 'l', 'h', 'g', 'e', 'z'
        ]),
    'stem-surface': random.choice([
        'unknown', 'y', 's', 't', 'g', 'h', 'k', 'i', 'f', 'l', 'd', 
        'x', 'w', 'a', 'o', 'c', 'n', 'm', 'e', 'p', 'z', 'b', 'r', 'u'
        ]),
    'stem-color': random.choice([
        'w', 'o', 'n', 'y', 'e', 'u', 'p', 'f', 'g', 'r', 'k', 'l', 
        'b', 'unknown', 't', 'z', 'a', 'h', 'd', 's', 'i', 'c', 'x', 'm'
        ]),
    'veil-type': random.choice([
        'unknown', 'u', 'd', 'a', 'h', 'g', 'c', 'e', 'y', 'i', 'f', 
        't', 'w', 'p', 'b', 's', 'k', 'r', 'l', 'n'
        ]),
    'veil-color': random.choice([
        'unknown', 'n', 'w', 'k', 'y', 'e', 'u', 'p', 'd', 'g', 'r', 
        'h', 's', 't', 'c', 'o', 'i', 'f', 'a', 'b', 'l', 'z'
        ]),
    'has-ring': random.choice([
        'f', 't', 'h', 'r', 'y', 'c', 'e', 'g', 'l', 's', 'unknown', 
        'p', 'x', 'k', 'z', 'd', 'o', 'n', 'm', 'i', 'w', 'a'
        ]),
    'ring-type': random.choice([
        'f', 'z', 'e', 'unknown', 'p', 'l', 'g', 'r', 'm', 'y', 'h', 
        'o', 't', 'a', 'd', 's', 'x', 'b', 'u', 'n', 'w', 'i', 'k', 'c'
        ]),
    'spore-print-color': random.choice([
        'unknown', 'k', 'w', 'p', 'n', 'r', 'u', 'g', 't', 'f', 'd', 
        'l', 'y', 'a', 's', 'e', 'o', 'c', 'b', 'h', 'x', 'i', 'm'
        ]),
    'habitat': random.choice([
        'd', 'l', 'g', 'h', 'p', 'm', 'u', 'w', 'y', 'unknown', 'n', 
        'a', 's', 'k', 'z', 'b', 't', 'c', 'e', 'r', 'f', 'o', 'x', 'i'
        ]),
    'season': random.choice([
        'a', 'w', 'u', 's'
        ]),
    'cap-diameter': random.uniform(0.03, 80.67),
    'stem-height': random.uniform(0.00, 88.72),
    'stem-width': random.uniform(0.00, 102.90),
}

In [7]:
import json
pd.read_json(json.dumps(data))

C:\Users\rbbel\AppData\Local\Temp\ipykernel_11300\2841883395.py:2: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  pd.read_json(json.dumps(data))


ValueError: If using all scalar values, you must pass an index

In [63]:
from tabulate import tabulate
tabulate(pd.DataFrame([data]), headers=pd.DataFrame([data]).columns, tablefmt='psql')

'+----+-------------+---------------+-------------+------------------------+-------------------+----------------+--------------+-------------+----------------+--------------+-------------+--------------+------------+-------------+---------------------+-----------+----------+----------------+---------------+--------------+\n|    | cap-shape   | cap-surface   | cap-color   | does-bruise-or-bleed   | gill-attachment   | gill-spacing   | gill-color   | stem-root   | stem-surface   | stem-color   | veil-type   | veil-color   | has-ring   | ring-type   | spore-print-color   | habitat   | season   |   cap-diameter |   stem-height |   stem-width |\n|----+-------------+---------------+-------------+------------------------+-------------------+----------------+--------------+-------------+----------------+--------------+-------------+--------------+------------+-------------+---------------------+-----------+----------+----------------+---------------+--------------|\n|  0 | m           | k     

In [49]:
df=pd.read_csv('data/train.csv')

In [29]:
df

,id,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,...,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
0,0,e,8.80,f,s,u,f,a,c,w,...,NaN,NaN,w,NaN,NaN,f,f,NaN,d,a
1,1,p,4.51,x,h,o,f,a,c,n,...,NaN,y,o,NaN,NaN,t,z,NaN,d,w
2,2,e,6.94,f,s,b,f,x,c,w,...,NaN,s,n,NaN,NaN,f,f,NaN,l,w
3,3,e,3.88,f,y,g,f,s,NaN,g,...,NaN,NaN,w,NaN,NaN,f,f,NaN,d,u
4,4,e,5.85,x,l,w,f,d,NaN,w,...,NaN,NaN,w,NaN,NaN,f,f,NaN,g,a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3116940,3116940,e,9.29,f,NaN,n,t,NaN,NaN,w,...,b,NaN,w,u,w,t,g,NaN,d,u
3116941,3116941,e,10.88,s,NaN,w,t,d,c,p,...,NaN,NaN,w,NaN,NaN,f,f,NaN,d,u
3116942,3116942,p,7.82,x,e,e,f,a,NaN,w,...,NaN,NaN,y,NaN,w,t,z,NaN,d,a
3116943,3116943,e,9.45,p,i,n,t,e,NaN,p,...,NaN,y,w,NaN,NaN,t,p,NaN,d,u


In [30]:
num = df.select_dtypes(exclude='object')

In [34]:
num.min()

id              0.00
cap-diameter    0.03
stem-height     0.00
stem-width      0.00
dtype: float64

In [32]:
for col in num.columns:
    print(f"'{col}': {num[col].unique().tolist()}")

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [ ]:
'cap-diameter': random.uniform(),
'stem-height': random.uniform(),
'stem-width': random.uniform(),


In [2]:
import pandas as pd

In [4]:
pd.read_csv("monitoring/metrics_app/datasets/mushroom/reference.csv").

,id,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,...,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
0,0,e,8.80,f,s,u,f,a,c,w,...,NaN,NaN,w,NaN,NaN,f,f,NaN,d,a
1,1,p,4.51,x,h,o,f,a,c,n,...,NaN,y,o,NaN,NaN,t,z,NaN,d,w
2,2,e,6.94,f,s,b,f,x,c,w,...,NaN,s,n,NaN,NaN,f,f,NaN,l,w
3,3,e,3.88,f,y,g,f,s,NaN,g,...,NaN,NaN,w,NaN,NaN,f,f,NaN,d,u
4,4,e,5.85,x,l,w,f,d,NaN,w,...,NaN,NaN,w,NaN,NaN,f,f,NaN,g,a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3116940,3116940,e,9.29,f,NaN,n,t,NaN,NaN,w,...,b,NaN,w,u,w,t,g,NaN,d,u
3116941,3116941,e,10.88,s,NaN,w,t,d,c,p,...,NaN,NaN,w,NaN,NaN,f,f,NaN,d,u
3116942,3116942,p,7.82,x,e,e,f,a,NaN,w,...,NaN,NaN,y,NaN,w,t,z,NaN,d,a
3116943,3116943,e,9.45,p,i,n,t,e,NaN,p,...,NaN,y,w,NaN,NaN,t,p,NaN,d,u
